# Create Area of Interest

**Timm Nawrocki**  
Alaska Center for Conservation Science  
2019-04-01

In [1]:
# -*- coding: utf-8 -*-
# ---------------------------------------------------------------------------
# Create Area of Interest
# Author: Timm Nawrocki
# Created on: 2019-04-01
# Usage: Must be executed as a Jupyter Notebook in an ArcGIS Pro Python 3 installation.
# Description: "Create Area of Interest" generates a raster representation of the model area that is limited by the maximum coverage of all input rasters and set to a common grid and cell size.

## 1. Initialize Environment

In [2]:
# Import packages
import arcpy
from arcpy.sa import *
import datetime
import os
import time

# Set overwrite option
arcpy.env.overwriteOutput = True

# Set root directory
drive = 'F:/'
root_directory = os.path.join(drive, 'ACCS_Work/Projects/VegetationEcology/BristolBay_Vegetation/Project_GIS')

# Set arcpy working environment
arcpy.env.workspace = os.path.join(root_directory, 'BristolBay_Vegetation.gdb')

# Define script paths
script_directory = os.path.join(drive, 'ACCS_Work/Repositories/southwest-alaska-moose/modules')
arcpy_geoprocessing_script = os.path.join(script_directory, 'arcpy_geoprocessing.py')

# Define input datasets
model_area = os.path.join(arcpy.env.workspace, 'SouthwestAlaska_ModelArea_Clipped')
elevation = os.path.join(root_directory, 'Data_Input/source_data/elevation/USGS3DEP_60m_DEM/Alaska_USGS3DEP_Elevation_60m_AKALB_20181106.tif')
compound_topographic = os.path.join(root_directory, 'Data_Input/source_data/elevation/USGS3DEP_60m_DEM/Alaska_Topography_CompoundTopographic_60m_AKALB_20181106.tif')
heat_load = os.path.join(root_directory, 'Data_Input/source_data/elevation/USGS3DEP_60m_DEM/Alaska_Topography_HeatLoad_60m_AKALB_20181106.tif')
integrated_moisture = os.path.join(root_directory, 'Data_Input/source_data/elevation/USGS3DEP_60m_DEM/Alaska_Topography_IntegratedMoisture_60m_AKALB_20181106.tif')
linear_aspect = os.path.join(root_directory, 'Data_Input/source_data/elevation/USGS3DEP_60m_DEM/Alaska_Topography_LinearAspect_60m_AKALB_20181106.tif')
mean_slope = os.path.join(root_directory, 'Data_Input/source_data/elevation/USGS3DEP_60m_DEM/Alaska_Topography_MeanSlope3x3_60m_AKALB_20181106.tif')
roughness = os.path.join(root_directory, 'Data_Input/source_data/elevation/USGS3DEP_60m_DEM/Alaska_Topography_Roughness3x3_60m_AKALB_20181106.tif')
site_exposure = os.path.join(root_directory, 'Data_Input/source_data/elevation/USGS3DEP_60m_DEM/Alaska_Topography_SiteExposure_60m_AKALB_20181106.tif')
surface_area_ratio = os.path.join(root_directory, 'Data_Input/source_data/elevation/USGS3DEP_60m_DEM/Alaska_Topography_SurfaceAreaRatio_60m_AKALB_20181106.tif')
surface_relief = os.path.join(root_directory, 'Data_Input/source_data/elevation/USGS3DEP_60m_DEM/Alaska_Topography_SurfaceRelief3x3_60m_AKALB_20181106.tif')
climate_representative = os.path.join(root_directory, 'Data_Input/source_data/climatology/Alaska_DateOfFreeze_2000s.tif')
imagery_representative = os.path.join(root_directory, 'Data_Input/source_data/imagery/sentinel2/Sent2_05May_1_ultraBlue.tif')

# Define output path and outputs
output_path = os.path.join(root_directory, 'Data_Input/area_of_interest')
area_of_interest = os.path.join(output_path, 'area_of_interest.tif')

In [3]:
# Import and execute arcpy_geoprocessing.py
try:
    exec(open(arcpy_geoprocessing_script).read())
except:
    print("Error loading arcpy_geoprocessing.py; ensure that script directory is correct:")
    print(script_directory)
    quit()

## 2. Create Area of Interest

In [4]:
# Define a function to create and refine an area of interest
def create_aoi(**kwargs):
    """
    Description: creates an area of interest (aoi) from a model area and iteratively extracts the aoi to the additional input rasters
    Inputs: 'input_array' -- an array containing the input datasets with the first dataset as the model area feature class, the second as the elevation raster, and any number of additional environmental rasters
            'output_array' -- an array containing the location to save the area of interest raster
    """
    # Start timing function execution
    start = time.time()
    # Parse key word argument inputs
    input_rasters = kwargs['input_array']
    model_area = input_rasters.pop(0)
    elevation = input_rasters[0]
    # Set the snap raster environment
    arcpy.env.snapRaster = elevation
    # Convert the model area polygon to a raster snapped to the USGS Elevation grid with a 10 m cell size
    print('Converting model area to raster...')
    arcpy.PolygonToRaster_conversion(model_area, 'OBJECTID', area_of_interest, 'CELL_CENTER', '', 10)
    # Set the cell size environment
    arcpy.env.cellSize = 10
    # Define a function to refine the initial area of interest based on the data extent of the unformatted predictor rasters
    def refineAOI(area, predictor):
        # Extract initial area of interest to predictor raster
        print('Refining area of interest...')
        extract_raster = ExtractByMask(area, predictor)
        arcpy.Delete_management(area)
        arcpy.CopyRaster_management(extract_raster, area, "", "", "", "NONE", "NONE", "1_BIT", "NONE", "NONE", "TIFF", "NONE")
    # Iterate the refine AOI function for each raster selected as an input
    for input_raster in input_rasters:
        refineAOI(area_of_interest, input_raster)
    # End timing function execution and calculate elapsed time
    end = time.time()
    elapsed = int(end - start)
    success_time = datetime.datetime.now()
    # Report process success
    print('Successfully created area of interest...')
    print('----------')
    out_process = 'Succeeded at {0} (Elapsed time: {1})'.format(success_time.strftime("%Y-%m-%d %H:%M"),
                                                                datetime.timedelta(seconds=elapsed))
    return out_process

In [5]:
# Define input and output arrays
aoi_inputs = [model_area,
              elevation,
              compound_topographic,
              heat_load,
              integrated_moisture,
              linear_aspect,
              mean_slope,
              roughness,
              site_exposure,
              surface_area_ratio,
              surface_relief,
              climate_representative,
              imagery_representative
             ]
aoi_outputs = [area_of_interest]

# Create key word arguments
aoi_kwargs = {'input_array' : aoi_inputs,
              'output_array' : aoi_outputs,
              'output_path' : output_path
             }

# Process the create polygon function with the point array
arcpy_geoprocessing(create_aoi, **aoi_kwargs)

Converting model area to raster...
Refining area of interest...
Refining area of interest...
Refining area of interest...
Refining area of interest...
Refining area of interest...
Refining area of interest...
Refining area of interest...
Refining area of interest...
Refining area of interest...
Refining area of interest...
Refining area of interest...
Refining area of interest...
Successfully created area of interest...
----------
Succeeded at 2019-04-02 14:10 (Elapsed time: 0:57:13)
